一共分为3个步骤

1.  stage1, 主aspect categories分类，需要增强数据
2.  stage2, 取得正确的主aspect category后，拼凑成完整的categories对，再用模型分类。
3.  stage3, 拼接categories对, 做情感极性分类。



# 预处理

In [1]:
!nvidia-smi

Wed Mar  8 12:52:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.106.00   Driver Version: 460.106.00   CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:08.0 Off |                    0 |
| N/A   36C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# 安装必要的包
!pip install transformers

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


*处理数据*

In [3]:
# 读取数据
# laptop 或者 res
import pandas as pd
data_name = "laptop"

data_raw = pd.read_json("dataJson/{}_train.json".format(data_name))



In [4]:
data_raw.head(10)

,sentence,category
0,acer wants $ 170 to just look at it then add t...,SUPPORT#PRICE
1,update : i repaired it myself for $ 12 .,LAPTOP#GENERAL
2,i had nothing to lose since it was a paper wei...,LAPTOP#GENERAL
3,the shame of it is knowing it took me 15 minut...,SUPPORT#GENERAL
4,first one that they shipped was obviously defe...,SHIPPING#GENERAL
5,first one that they shipped was obviously defe...,SHIPPING#GENERAL
6,first one that they shipped was obviously defe...,MULTIMEDIA_DEVICES#GENERAL
7,"the replacement i got was much better , but st...",LAPTOP#QUALITY
8,i wound up returning it .,LAPTOP#GENERAL
9,this works fine for that .,LAPTOP#QUALITY


In [5]:
categories_list = []
for item in data_raw["category"]:
  if item not in categories_list:
    categories_list.append(item)


In [6]:
categories_list

['SUPPORT#PRICE',
 'LAPTOP#GENERAL',
 'SUPPORT#GENERAL',
 'SHIPPING#GENERAL',
 'MULTIMEDIA_DEVICES#GENERAL',
 'LAPTOP#QUALITY',
 'PORTS#QUALITY',
 'HARDWARE#GENERAL',
 'LAPTOP#DESIGN_FEATURES',
 'LAPTOP#OPERATION_PERFORMANCE',
 'DISPLAY#OPERATION_PERFORMANCE',
 'LAPTOP#PRICE',
 'KEYBOARD#QUALITY',
 'KEYBOARD#OPERATION_PERFORMANCE',
 'POWER_SUPPLY#QUALITY',
 'POWER_SUPPLY#GENERAL',
 'BATTERY#QUALITY',
 'POWER_SUPPLY#OPERATION_PERFORMANCE',
 'DISPLAY#DESIGN_FEATURES',
 'LAPTOP#PORTABILITY',
 'HARDWARE#OPERATION_PERFORMANCE',
 'SOFTWARE#GENERAL',
 'OS#QUALITY',
 'HARDWARE#QUALITY',
 'DISPLAY#GENERAL',
 'FANS&COOLING#GENERAL',
 'FANS&COOLING#QUALITY',
 'KEYBOARD#GENERAL',
 'OS#OPERATION_PERFORMANCE',
 'CPU#PRICE',
 'KEYBOARD#PRICE',
 'BATTERY#GENERAL',
 'BATTERY#OPERATION_PERFORMANCE',
 'Out_Of_Scope#GENERAL',
 'MULTIMEDIA_DEVICES#OPERATION_PERFORMANCE',
 'PORTS#OPERATION_PERFORMANCE',
 'LAPTOP#CONNECTIVITY',
 'CPU#GENERAL',
 'OS#GENERAL',
 'MEMORY#GENERAL',
 'DISPLAY#QUALITY',
 'COMPANY#G

In [7]:
# # 切分开entity category 和 attibute, 也就是以后要用的两个阶段的category
# categories_list_stage1 = []
# categories_list_stage2 = []
# for item in categories_list:
#   category1, category2 = item.split('#')
#   if category1 not in categories_list_stage1:
#     categories_list_stage1.append(category1)
#   if category2 not in categories_list_stage2:
#     categories_list_stage2.append(category2)

  

In [8]:
# print("stage one has {} categories. ".format(len(categories_list_stage1)))
# print("stage two has {} categories. ".format(len(categories_list_stage2)))

In [9]:
# print("stage1 categories:", categories_list_stage1)
# print("stage2 categories:", categories_list_stage2)

In [10]:
len(categories_list)

114

## *数据增强*


1.   data_stage1
2.   列表项



In [11]:
# data_raw.drop(columns=["aspect_span", "opinion_span", "tags"])

In [12]:
# 每个sentence 遍历所有的stage1_categories, 构造成[setence, category, "yes, no"]


data_stage1 = []

for row in data_raw.itertuples():
  data_category = row.category
  for category in categories_list:
    YES = 0
    if data_category == category:
      YES = 1
      for i in range(20):
        data_stage1.append({"sentence":row.sentence, "category":category.split('#')[0] + " "+ category.split('#')[1], "YES":YES})
    
    data_stage1.append({"sentence":row.sentence, "category":category.split('#')[0] + " "+ category.split('#')[1], "YES":YES})

    


In [13]:
data_stage1[0:5]
len(data_stage1)

559048

In [14]:
# 划分数据
import torch
from torch.utils.data import random_split

train_size = int(len(data_stage1) * 0.8)
eval_size = int(len(data_stage1) * 0.1)
test_size = len(data_stage1) - train_size - eval_size

train_stage1, eval_stage1, test_stage1 = random_split(data_stage1, [train_size, eval_size, test_size], generator=torch.Generator().manual_seed(42))

In [15]:
for index, line in enumerate(train_stage1):
  if index <= 9:
    print(line["sentence"])

my laptop is affected by the staingate issue and apple denied responsibility , saying that it ' s a cosmetic problem caused by improper cleaning , yet there are thousands of reports on this .
this is my third and last chromebook .
the mousepad was not very responsive .
awesome display .
also , chromeos does not allow color / temperature calibration of the display device .
excellent chromebooks !
the keyboard is a delight with large , nearly flush , keys set to just the right level of resistance and sensitivity .
i packaged it right back up and sent it back the same day i got it .
asus support is horrible .
fan noise could be a bit quieter when the cpu is being taxed but not annoyingly loud either , under lite use the pc is silent .


In [16]:
!pwd

/home/ubuntu/zhangmao/ACD


# *stage1*

1.   data augmented
2.   列表项



## *配置*

In [17]:
# coding: UTF-8
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import RobertaModel, BertTokenizer
from transformers import AutoTokenizer, AutoModelForMaskedLM


class Config(object):

    """配置参数"""
    def __init__(self, dataset):
        self.model_name = 'Robert'
        # self.train_path = dataset + '/data/train.txt'                                # 训练集
        # self.dev_path = dataset + '/data/dev.txt'                                    # 验证集
        # self.test_path = dataset + '/data/test.txt'                                  # 测试集
        self.class_list = ['NO', 'YES']                                # 类别名单
        self.save_path = dataset + '/saved_dict/' + data_name + self.model_name + '.ckpt'        # 模型训练结果
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   # 设备
     
        self.require_improvement = 1000                                 # 若超过1000batch效果还没提升，则提前结束训练
        self.num_classes = len(self.class_list)                         # 类别数
        self.num_epochs = 3                                            # epoch数
        self.batch_size = 32                                           # mini-batch大小
        self.pad_size = 64                                              # 每句话处理成的长度(短填长切)
        self.learning_rate = 5e-5                                       # 学习率
        self.pretrained_model = 'xlm-roberta-base'
        self.bert_path = 'xlm-roberta-base'
        self.tokenizer = AutoTokenizer.from_pretrained(self.pretrained_model)
        self.hidden_size = 768                                       # 卷积核数量(channels数)
        self.dropout = 0.1

## *数据集tokenize*

In [18]:
# build dataset
import torch
# coding: UTF-8
import torch
from tqdm import tqdm
import time
from datetime import timedelta

PAD, CLS = '[PAD]', '[CLS]'  # padding符号, bert中综合信息符号
SEP = '[SEP]'

def build_dataset_stage1(config):

    def load_dataset(data_stage1_, pad_size=32):
        contents = []
        
        for item in tqdm(data_stage1_):
            if not item:
                continue
            content, category, label = item['sentence'], item['category'], item['YES']
            content_token = config.tokenizer.tokenize(content)
            category_token = config.tokenizer.tokenize(category)
            token = [CLS] + content_token + [SEP] + category_token
            seq_len = len(token)
            mask = []
            token_ids = config.tokenizer.convert_tokens_to_ids(token)

            if pad_size:
                if len(token) < pad_size:
                    mask = [1] * len(token_ids) + [0] * (pad_size - len(token))
                    token_ids += ([0] * (pad_size - len(token)))
                else:
                    mask = [1] * pad_size
                    token_ids = token_ids[:pad_size]
                    seq_len = pad_size
            contents.append((token_ids, int(label), seq_len, mask))
        return contents
    train = load_dataset(train_stage1, config.pad_size)
    dev = load_dataset(eval_stage1, config.pad_size)
    test = load_dataset(test_stage1, config.pad_size)
    return train, dev, test


In [19]:
config = Config('./')
tk_train_stage1, tk_dev_stage1, tk_test_stage1 = build_dataset_stage1(config)

100%|████████████████████████████████████████████████████████████████████████████████| 55906/55906 [00:10<00:00, 5230.04it/s]


In [20]:
class DatasetIterater(object):
    def __init__(self, batches, batch_size, device):
        self.batch_size = batch_size
        self.batches = batches
        self.n_batches = len(batches) // batch_size
        self.residue = False  # 记录batch数量是否为整数
        if len(batches) % self.n_batches != 0:
            self.residue = True
        self.index = 0
        self.device = device

    def _to_tensor(self, datas):
        x = torch.LongTensor([_[0] for _ in datas]).to(self.device)
        y = torch.LongTensor([_[1] for _ in datas]).to(self.device)

        # pad前的长度(超过pad_size的设为pad_size)
        seq_len = torch.LongTensor([_[2] for _ in datas]).to(self.device)
        mask = torch.LongTensor([_[3] for _ in datas]).to(self.device)
        return (x, seq_len, mask), y

    def __next__(self):
        if self.residue and self.index == self.n_batches:
            batches = self.batches[self.index * self.batch_size: len(self.batches)]
            self.index += 1
            batches = self._to_tensor(batches)
            return batches

        elif self.index >= self.n_batches:
            self.index = 0
            raise StopIteration
        else:
            batches = self.batches[self.index * self.batch_size: (self.index + 1) * self.batch_size]
            self.index += 1
            batches = self._to_tensor(batches)
            return batches

    def __iter__(self):
        return self

    def __len__(self):
        if self.residue:
            return self.n_batches + 1
        else:
            return self.n_batches


def build_iterator(dataset, config):
    iter = DatasetIterater(dataset, config.batch_size, config.device)
    return iter


def get_time_dif(start_time):
    """获取已使用时间"""
    end_time = time.time()
    time_dif = end_time - start_time
    return timedelta(seconds=int(round(time_dif)))


## *模型结构*

In [21]:
# base bert
# base bert


import torch.nn.functional as F

unfreeze_layers = ['layer.8', 'layer.9', 'layer.11', 'bert.pooler', 'out.']


class Model(nn.Module):

    def __init__(self, config):
        super(Model, self).__init__()
        self.bert = RobertaModel.from_pretrained(config.bert_path)
        
#         所有参数
#         for param in self.bert.parameters():
#             param.requires_grad = True
        
#      冻结一部分参数
        for name, param in self.bert.named_parameters():
            param.requires_grad = False
            for ele in unfreeze_layers:
                if ele in name:
                    param.requires_grad = True
     
        self.dropout = nn.Dropout(config.dropout)

        self.linear = nn.Linear(config.hidden_size, config.num_classes)
        self.relu = F.relu
        



    def forward(self, x):
        context = x[0]  # 输入的句子
        mask = x[2]  # 对padding部分进行mask，和句子一个size，padding部分用0表示，如：[1, 1, 1, 1, 0, 0]
        output = self.bert(context, attention_mask=mask)


        # f = open("bertoutput.txt", "a")
        # f.write(str(output))
        # f.close()


        
        pooled_output = output.last_hidden_state[:, 0, :]

        # out = encoder_out.unsqueeze(1)
        out = self.dropout(pooled_output)
        out = self.linear(out)
        out = self.relu(out)

        # out = self.relu(out)
        # F.cross_entropy 自带softmax        

        return out

## *训练方法*

In [22]:
# coding: UTF-8
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
import time
# from python_utils import get_time_dif
# from transformers.optimization import AdamW    de

from torch.optim import AdamW


# 权重初始化，默认xavier
def init_network(model, method='xavier', exclude='embedding', seed=123):
    for name, w in model.named_parameters():
        if exclude not in name:
            if len(w.size()) < 2:
                continue
            if 'weight' in name:
                if method == 'xavier':
                    nn.init.xavier_normal_(w)
                elif method == 'kaiming':
                    nn.init.kaiming_normal_(w)
                else:
                    nn.init.normal_(w)
            elif 'bias' in name:
                nn.init.constant_(w, 0)
            else:
                pass


def train(config, model, train_iter, dev_iter, test_iter):
    start_time = time.time()
    model.train()
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
    # optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    optimizer = AdamW(optimizer_grouped_parameters,
                         lr=config.learning_rate,
                         )
    total_batch = 0  # 记录进行到多少batch
    dev_best_loss = float('inf')
    last_improve = 0  # 记录上次验证集loss下降的batch数
    flag = False  # 记录是否很久没有效果提升
    model.train() # 打印出相关训练信息


    for epoch in range(config.num_epochs):
        print('Epoch [{}/{}]'.format(epoch + 1, config.num_epochs))
        for i, (trains, labels) in enumerate(train_iter):
            outputs = model(trains)
            # print("model outputs:", outputs)
            model.zero_grad()
            loss = F.cross_entropy(outputs, labels)
            loss.backward()
            optimizer.step()
            if total_batch % 100 == 0:
                # 每多少轮输出在训练集和验证集上的效果
                true = labels.data.cpu()
                predic = torch.max(outputs.data, 1)[1].cpu()
                train_acc = metrics.accuracy_score(true, predic)
                dev_acc, dev_loss = evaluate(config, model, dev_iter)
                if dev_loss < dev_best_loss:
                    dev_best_loss = dev_loss
                    torch.save(model.state_dict(), config.save_path) # 保存模型
                    improve = '*'
                    last_improve = total_batch
                else:
                    improve = ''
                time_dif = get_time_dif(start_time)
                msg = 'Iter: {0:>6},  Train Loss: {1:>5.2},  Train Acc: {2:>6.2%},  Val Loss: {3:>5.2},  Val Acc: {4:>6.2%},  Time: {5} {6}'
                print(msg.format(total_batch, loss.item(), train_acc, dev_loss, dev_acc, time_dif, improve))
                model.train()
            total_batch += 1
            if total_batch - last_improve > config.require_improvement:
                # 验证集loss超过1000batch没下降，结束训练
                print("No optimization for a long time, auto-stopping...")
                flag = True
                break
        if flag:
            break
    test(config, model, test_iter)


def test(config, model, test_iter):
    # test
    model.load_state_dict(torch.load(config.save_path))
    model.eval()
    start_time = time.time()
    test_acc, test_loss, test_report, test_confusion = evaluate(config, model, test_iter, test=True)
    msg = 'Test Loss: {0:>5.2},  Test Acc: {1:>6.2%}'
    print(msg.format(test_loss, test_acc))
    print("Precision, Recall and F1-Score...")
    print(test_report)
    print("Confusion Matrix...")
    print(test_confusion)
    # 替换原来的get_time_dif
    end_time = time.time()
    time_dif = end_time - start_time
    print("Time usage:", time_dif)


def evaluate(config, model, data_iter, test=False):
    model.eval()
    loss_total = 0
    predict_all = np.array([], dtype=int)
    labels_all = np.array([], dtype=int)
    with torch.no_grad():
        for texts, labels in data_iter:
            outputs = model(texts)
            loss = F.cross_entropy(outputs, labels)
            loss_total += loss
            labels = labels.data.cpu().numpy()
            predic = torch.max(outputs.data, 1)[1].cpu().numpy()
            labels_all = np.append(labels_all, labels)
            predict_all = np.append(predict_all, predic)

    acc = metrics.accuracy_score(labels_all, predict_all)
    if test:
        report = metrics.classification_report(labels_all, predict_all, target_names=config.class_list, digits=4)
        confusion = metrics.confusion_matrix(labels_all, predict_all)
        return acc, loss_total / len(data_iter), report, confusion
    return acc, loss_total / len(data_iter)


## *开始训练*

In [23]:
# 训练
# coding: UTF-8
import time
import torch
import numpy as np

from importlib import import_module
import argparse

dataset = "./"  # 数据集

config = Config(dataset)
np.random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed_all(1)
torch.backends.cudnn.deterministic = True #保证每一次的结果一样


start_time = time.time()
print("Loading data...")
train_data, dev_data, test_data = build_dataset_stage1(config)
train_iter = build_iterator(train_data, config)
dev_iter = build_iterator(dev_data, config)
test_iter = build_iterator(test_data, config)
time_dif = get_time_dif(start_time)
print("Time usage:", time_dif)


Loading data...


100%|████████████████████████████████████████████████████████████████████████████████| 55906/55906 [00:10<00:00, 5149.14it/s]

Time usage: 0:01:50


In [24]:
model = Model(config).to(config.device)
train(config, model, train_iter, dev_iter, test_iter)

You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch [1/3]
Iter:      0,  Train Loss:  0.99,  Train Acc: 12.50%,  Val Loss:  0.86,  Val Acc: 18.03%,  Time: 0:03:19 *
Iter:    100,  Train Loss:  0.34,  Train Acc: 87.50%,  Val Loss:  0.39,  Val Acc: 84.27%,  Time: 0:07:15 *
Iter:    200,  Train Loss:   0.4,  Train Acc: 78.12%,  Val Loss:  0.32,  Val Acc: 84.27%,  Time: 0:11:13 *
Iter:    300,  Train Loss:   0.5,  Train Acc: 81.25%,  Val Loss:   0.3,  Val Acc: 84.27%,  Time: 0:15:10 *
Iter:    400,  Train Loss:  0.31,  Train Acc: 68.75%,  Val Loss:  0.29,  Val Acc: 84.27%,  Time: 0:19:08 *
Iter:    500,  Train Loss:  0.22,  Train Acc: 84.38%,  Val Loss:  0.28,  Val Acc: 84.27%,  Time: 0:23:05 *
Iter:    600,  Train Loss:   0.2,  Train Acc: 84.38%,  Val Loss:  0.29,  Val Acc: 84.27%,  Time: 0:26:54 
Iter:    700,  Train Loss:   0.2,  Train Acc: 93.75%,  Val Loss:  0.28,  Val Acc: 84.27%,  Time: 0:30:44 
Iter:    800,  Train Loss:  0.65,  Train Acc: 65.62%,  Val Loss:  0.26,  Val Acc: 84.27%,  Time: 0:34:43 *
Iter:    900,  Train Loss: 

Iter:   7700,  Train Loss:  0.24,  Train Acc: 87.50%,  Val Loss:  0.13,  Val Acc: 94.94%,  Time: 12:38:28 
Iter:   7800,  Train Loss:  0.12,  Train Acc: 96.88%,  Val Loss:  0.13,  Val Acc: 94.81%,  Time: 12:50:35 
Iter:   7900,  Train Loss:  0.19,  Train Acc: 93.75%,  Val Loss:  0.12,  Val Acc: 95.49%,  Time: 13:02:50 *
Iter:   8000,  Train Loss: 0.053,  Train Acc: 96.88%,  Val Loss:  0.12,  Val Acc: 95.23%,  Time: 13:14:55 
Iter:   8100,  Train Loss:  0.16,  Train Acc: 87.50%,  Val Loss:  0.11,  Val Acc: 95.45%,  Time: 13:27:09 *
Iter:   8200,  Train Loss: 0.062,  Train Acc: 100.00%,  Val Loss:  0.13,  Val Acc: 94.92%,  Time: 13:39:14 
Iter:   8300,  Train Loss:  0.16,  Train Acc: 93.75%,  Val Loss:  0.12,  Val Acc: 95.39%,  Time: 13:51:24 
Iter:   8400,  Train Loss:  0.18,  Train Acc: 93.75%,  Val Loss:  0.12,  Val Acc: 95.45%,  Time: 14:03:27 
Iter:   8500,  Train Loss:  0.18,  Train Acc: 90.62%,  Val Loss:  0.11,  Val Acc: 95.59%,  Time: 14:15:40 *
Iter:   8600,  Train Loss:  0.41,

In [25]:
# torch.save(model.state_dict(),'rest_roberta_ACD.pth')

## stage1 *功能测试*

In [31]:
testdata_ = test_stage1[10]
testdata_  

{'sentence': "the pricing was too good to pass up and i ' ve really been pleased with the product .",
 'category': 'KEYBOARD DESIGN_FEATURES',
 'YES': 0}

In [32]:
from transformers import AutoTokenizer
# test_sentence = "I found it is hard to use."

test_sentence = testdata_["sentence"]


tokenizer = AutoTokenizer.from_pretrained(config.pretrained_model)
pad_size = 64

pred_category = []
for category in categories_list:
  s_token = tokenizer.tokenize(test_sentence)
  category_token = tokenizer.tokenize(category)
  token = [CLS] + s_token + [SEP] + category_token
  seq_len = len(token)
  mask = []
  token_ids = tokenizer.convert_tokens_to_ids(token)
  print(token_ids)

  if pad_size:
      if len(token) < pad_size:
          mask = [1] * len(token_ids) + [0] * (pad_size - len(token))
          token_ids += ([0] * (pad_size - len(token)))
      else:
          mask = [1] * pad_size
          token_ids = token_ids[:pad_size]
          seq_len = pad_size

          # to tensor 



  token_ids = torch.unsqueeze(torch.tensor(token_ids, dtype=torch.long), dim=0).to(config.device)
  mask = torch.unsqueeze(torch.tensor(mask), dim=0).to(config.device)
  seq_len = torch.unsqueeze(torch.tensor(seq_len, dtype=torch.long), dim=0).to(config.device)
  # print(token_ids, seq_len, mask)
  
  output = model((token_ids, seq_len, mask))
  print(output)
  output = F.softmax(output, dim=1)
    
    

  print(test_sentence, 'category:', category)
  print("is match? : ", output.argmax(dim=1).cpu())
  if output.argmax(dim=1).cpu() == 1:
    pred_category.append(category)



[3, 70, 494, 21896, 509, 5792, 4127, 47, 27875, 1257, 136, 17, 242, 173, 6183, 2809, 174849, 678, 70, 12996, 6, 5, 3, 161463, 91599, 4904, 17255, 31722]
tensor([[9.5203, 0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)
the pricing was too good to pass up and i ' ve really been pleased with the product . category: SUPPORT#PRICE
is match? :  tensor([0])
[3, 70, 494, 21896, 509, 5792, 4127, 47, 27875, 1257, 136, 17, 242, 173, 6183, 2809, 174849, 678, 70, 12996, 6, 5, 3, 6, 85324, 70675, 4904, 724, 88078, 7842]
tensor([[2.1428, 0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)
the pricing was too good to pass up and i ' ve really been pleased with the product . category: LAPTOP#GENERAL
is match? :  tensor([0])
[3, 70, 494, 21896, 509, 5792, 4127, 47, 27875, 1257, 136, 17, 242, 173, 6183, 2809, 174849, 678, 70, 12996, 6, 5, 3, 161463, 91599, 4904, 724, 88078, 7842]
tensor([[9.8881, 0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)
the pricing was too good to pass up and i ' ve really

tensor([[9.8132, 0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)
the pricing was too good to pass up and i ' ve really been pleased with the product . category: HARDWARE#QUALITY
is match? :  tensor([0])
[3, 70, 494, 21896, 509, 5792, 4127, 47, 27875, 1257, 136, 17, 242, 173, 6183, 2809, 174849, 678, 70, 12996, 6, 5, 3, 209973, 103695, 4904, 724, 88078, 7842]
tensor([[9.8743, 0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)
the pricing was too good to pass up and i ' ve really been pleased with the product . category: DISPLAY#GENERAL
is match? :  tensor([0])
[3, 70, 494, 21896, 509, 5792, 4127, 47, 27875, 1257, 136, 17, 242, 173, 6183, 2809, 174849, 678, 70, 12996, 6, 5, 3, 563, 48789, 1230, 9688, 670, 45134, 4904, 724, 88078, 7842]
tensor([[9.9590, 0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)
the pricing was too good to pass up and i ' ve really been pleased with the product . category: FANS&COOLING#GENERAL
is match? :  tensor([0])
[3, 70, 494, 21896, 509, 5792, 4127, 47,

tensor([[8.8731, 0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)
the pricing was too good to pass up and i ' ve really been pleased with the product . category: WARRANTY#GENERAL
is match? :  tensor([0])
[3, 70, 494, 21896, 509, 5792, 4127, 47, 27875, 1257, 136, 17, 242, 173, 6183, 2809, 174849, 678, 70, 12996, 6, 5, 3, 72804, 54259, 102198, 4904, 670, 30425, 84780, 454, 30425, 98533, 113786]
tensor([[9.9199, 0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)
the pricing was too good to pass up and i ' ve really been pleased with the product . category: GRAPHICS#OPERATION_PERFORMANCE
is match? :  tensor([0])
[3, 70, 494, 21896, 509, 5792, 4127, 47, 27875, 1257, 136, 17, 242, 173, 6183, 2809, 174849, 678, 70, 12996, 6, 5, 3, 11292, 78633, 1723, 4904, 670, 30425, 84780, 454, 30425, 98533, 113786]
tensor([[9.9411, 0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)
the pricing was too good to pass up and i ' ve really been pleased with the product . category: MEMORY#OPERATION_PERFORMA

tensor([[9.9430, 0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)
the pricing was too good to pass up and i ' ve really been pleased with the product . category: SUPPORT#QUALITY
is match? :  tensor([0])
[3, 70, 494, 21896, 509, 5792, 4127, 47, 27875, 1257, 136, 17, 242, 173, 6183, 2809, 174849, 678, 70, 12996, 6, 5, 3, 11292, 78633, 1723, 4904, 120572, 866, 63823]
tensor([[10.0008,  0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)
the pricing was too good to pass up and i ' ve really been pleased with the product . category: MEMORY#QUALITY
is match? :  tensor([0])
[3, 70, 494, 21896, 509, 5792, 4127, 47, 27875, 1257, 136, 17, 242, 173, 6183, 2809, 174849, 678, 70, 12996, 6, 5, 3, 11292, 78633, 1723, 4904, 8762, 201526, 27143]
tensor([[9.9192, 0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)
the pricing was too good to pass up and i ' ve really been pleased with the product . category: MEMORY#USABILITY
is match? :  tensor([0])
[3, 70, 494, 21896, 509, 5792, 4127, 47, 27875, 125

tensor([[9.8981, 0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)
the pricing was too good to pass up and i ' ve really been pleased with the product . category: SOFTWARE#QUALITY
is match? :  tensor([0])
[3, 70, 494, 21896, 509, 5792, 4127, 47, 27875, 1257, 136, 17, 242, 173, 6183, 2809, 174849, 678, 70, 12996, 6, 5, 3, 6, 207776, 1456, 23711, 4904, 91599, 201526, 27143]
tensor([[9.8868, 0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)
the pricing was too good to pass up and i ' ve really been pleased with the product . category: SOFTWARE#PORTABILITY
is match? :  tensor([0])
[3, 70, 494, 21896, 509, 5792, 4127, 47, 27875, 1257, 136, 17, 242, 173, 6183, 2809, 174849, 678, 70, 12996, 6, 5, 3, 276, 224750, 17398, 49444, 4904, 120572, 866, 63823]
tensor([[9.9545, 0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)
the pricing was too good to pass up and i ' ve really been pleased with the product . category: MOTHERBOARD#QUALITY
is match? :  tensor([0])
[3, 70, 494, 21896, 509, 5792, 

In [33]:
pred_category

['LAPTOP#PRICE']